# Brain Imaging Data Structure (BIDS) tutorial on data conversion

In the following Jupyter notebook, we will introduce you to BIDS basics. You will download some phantom data in neuroimaging formats as they come of a scanner and then work on converting it to BIDS format.

## Collect the Repository and Install all Dependencies
This may take some time.

In [1]:
# Cross-platform repo setup (Python only, no shell commands)
import os
import sys
import subprocess

# Clone outreach repo if not present
repo_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'outreach'))
if not os.path.exists(repo_path):
    print('Cloning Outreach Repository...')
    subprocess.run(['git', 'clone', 'https://github.com/openneuropet/outreach.git', repo_path])
else:
    print('Repo already cloned. Pulling latest changes...')
    subprocess.run(['git', '-C', repo_path, 'pull'])

# Change directory to PETBIDS-Onboarding2025 if needed
target_dir = os.path.join(repo_path, 'PETBIDS-Onboarding2025')
if os.path.basename(os.getcwd()) != 'PETBIDS-Onboarding2025':
    os.chdir(target_dir)
    print(f'Changed working directory to {os.getcwd()}')

Repo already cloned. Pulling latest changes...


## How we will inspect our data  

In [2]:
# List files/folders in a directory (cross-platform, no tree or shell commands)
def list_directory(path, file_limit=25):
    if os.path.exists(path):
        print(f'Contents of {path}:')
        for i, item in enumerate(os.listdir(path)):
            print(item)
            if i >= file_limit - 1:
                print('... (file limit reached)')
                break
    else:
        print(f'Directory not found: {path}')

# Example usage: list files in current working directory
list_directory(os.getcwd())

Contents of c:\Users\spn486\Documents\GitHub\outreach\PETBIDS-Onboarding2025:
BIDS_conversion_tutorial_Colab.ipynb
BIDS_PET.png
PET2BIDS.pdf
PETprocessing
README.md
workflows_apps_derivatives_onboarding_stockholm_2025.pdf


## Check for dcm2niix and install if it's not present.

Additionally, we tell pypet2bids where the dcm2niix executable is located at, this is best practice on windows and any sort of virtual environment/notebook as the $PATH variable can be "wonky" in the later case.

In [ ]:
# Check for dcm2niix and install if not present (cross-platform)
import platform
import shutil
import subprocess
import sys

def find_dcm2niix():
    exe_name = 'dcm2niix.exe' if platform.system() == 'Windows' else 'dcm2niix'
    path = shutil.which(exe_name)
    if path:
        print(f'dcm2niix is installed at {path}')
        version = subprocess.run([path, '--version'], capture_output=True, text=True).stdout
        print(version)
        return path
    else:
        print('dcm2niix is not installed.')
        if platform.system() == 'Windows':
            print('Install with Chocolatey: choco install dcm2niix, or download from https://github.com/rordenlab/dcm2niix/releases and add to PATH.')
        elif platform.system() == 'Linux':
            print('Install with: sudo apt-get install dcm2niix, or download from https://github.com/rordenlab/dcm2niix/releases')
        elif platform.system() == 'Darwin':
            print('Install with: brew install dcm2niix, or download from https://github.com/rordenlab/dcm2niix/releases')
        else:
            print('Please install dcm2niix manually for your OS.')
        return None
    




dcm2niix_path = find_dcm2niix()

dcm2niix is installed at C:\Users\spn486\Anaconda3\Library\bin\dcm2niix.exe
Chris Rorden's dcm2niiX version v1.0.20211006  MSC1916  (64-bit Windows)
v1.0.20211006



## Install pypet2bids 

In [23]:
# install pypet2bids using pip if not already installed
try:    
    import pypet2bids
    print('pypet2bids is already installed.')   
except ImportError:
    print('Installing pypet2bids...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pypet2bids'])
    import pypet2bids
    print('pypet2bids installed successfully.')


pypet2bids is already installed.


In [21]:
!dcm2niix4pet 

usage: dcm2niix4pet [-h] [--metadata-path [METADATA_PATH]]
                    [--translation-script-path TRANSLATION_SCRIPT_PATH]
                    [--destination-path DESTINATION_PATH] [--tempdir TEMPDIR]
                    [--kwargs [KWARGS [KWARGS ...]]] [--silent]
                    [--show-examples] [--set-dcm2niix-path SET_DCM2NIIX_PATH]
                    [--set-default-metadata-json SET_DEFAULT_METADATA_JSON]
                    [--trc TRC] [--run RUN] [--rec REC]
                    [folder]
version: 1.3.9


In [22]:
# import other relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

## Setup the bids-validator
This is the easiest way to get the bids validator running on a Colab Notebook, from here on out it can be called with `bids-validator()`

In [7]:
def bids_validator(path_to_dataset='.'):
  validator = subprocess.run(f"/root/.deno/bin/deno run --allow-read --allow-env https://deno.land/x/bids_validator/bids-validator.ts {path_to_dataset}", shell=True, capture_output=True, text=True)
  print(validator.stdout)

bids_validator('')

## OpenNeuroPET Phantom Data

In [8]:
# Define where you have downloaded the phantom data (edit this path as needed for your system)

# Example: Windows path (raw string recommended)
data_dir = r"C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms"

# Example: Mac/Linux path (uncomment and edit if needed)
# data_dir = "/home/username/Downloads/PN000001/OpenNeuroPET-Phantoms"

# List contents of phantom data directory
list_directory(data_dir)

Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms:
code
dataset_description.json
LICENSE
mynewfolder
README
sourcedata
sub-CanonCartesionPrimeNIA
sub-GeneralElectricAdvanceJHU
sub-GeneralElectricAdvanceLongNIMH
sub-GeneralElectricAdvanceNIMH
sub-GeneralElectricDiscoveryAarhus
sub-GeneralElectricSignaAarhus
sub-GeneralElectricSignaNIMH
sub-PhilipsGeminiUnimedizinMainz
sub-PhilipsIngenuityPETCTAmsterdamUMC
sub-PhilipsIngenuityPETMRAmsterdamUMC
sub-PhillipsVereosAmsterdamUMC
sub-SiemensBiographNIMH
sub-SiemensBiographNRU
sub-SiemensHRRTJHU
sub-SiemensHRRTNRU
sub-SiemensTrioBrainPETFZJ


## Take a quick look at the raw dicom and ecat data that we've unzipped into this project folder.

In [9]:
# List files in the sourcedata subdirectory of the phantom data directory (cross-platform)

sourcedata_dir = os.path.join(data_dir, 'sourcedata')
list_directory(sourcedata_dir)

Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata:
CanonCartesionPrimePETCT-NIA.zip
CPSInnovationsHRRT-NIMH.zip
GeneralElectricAdvance-JHU.zip
GeneralElectricAdvance-NIMH.zip
GeneralElectricDiscoveryPETCT-Aarhus.zip
GeneralElectricSignaPETMR-Aarhus.zip
GeneralElectricSignaPETMR-NIMH.zip
PhilipsGeminiPETMR-Unimedizin.zip
PhilipsIngenuityPETCT-AmsterdamUMC.zip
PhilipsIngenuityPETMR-AmsterdamUMC.zip
PhillipsVereosPETCT-AmsterdamUMC.zip
SiemensBiographPETMR-NIMH
SiemensBiographPETMR-NIMH.zip
SiemensBiographPETMR-NRU
SiemensBiographPETMR-NRU.zip
SiemensHRRT-JHU.zip
SiemensHRRT-NRU.zip
SiemensMagnetomTrioBrainPET-FZJ.zip


Now we will unzip one of the phantom source data folders, namely the SiemensBiographPETMR-NRU folder, and take a look at the contents.

In [10]:
# Unzip the subfolders SiemensBiographPETMR-NRU.zip and SiemensBiographPETMR-NIMH.zip in the sourcedata folder (cross-platform)
import zipfile
nruf_zip_path = os.path.join(sourcedata_dir, 'SiemensBiographPETMR-NRU.zip')
nruf_extract_path = os.path.join(sourcedata_dir, 'SiemensBiographPETMR-NRU')
nimh_zip_path = os.path.join(sourcedata_dir, 'SiemensBiographPETMR-NIMH.zip')
nimh_extract_path = os.path.join(sourcedata_dir, 'SiemensBiographPETMR-NIMH')
for zip_path, extract_path in [(nruf_zip_path, nruf_extract_path), (nimh_zip_path, nimh_extract_path)]:
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f'Extracted {zip_path} to {extract_path}')
    else:
        print(f'Zip file not found: {zip_path}')

# List files in both extracted directories (cross-platform)
list_directory(nruf_extract_path)
list_directory(nimh_extract_path)

Extracted C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NRU.zip to C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NRU
Extracted C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NIMH.zip to C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NIMH
Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NRU:
SiemensBiographPETMR-NRU
SiemensBiographPETMR-NRU_Phantom_PetAcquisition_20220421120447_30003.json
SiemensBiographPETMR-NRU_Phantom_PetAcquisition_20220421120447_30003.nii
SiemensBiographPETMR-NRU_Phantom_PetAcquisition_20220421120447_30003a.json
SiemensBiographPETMR-NRU_Phantom_PetAcquisition_20220421120447_30003a.nii
Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NIMH:
AC_TOF
CT
NAC
SiemensBiographPETMR-NIMH
Extracted C:\Users

## Examine and convert some dicoms obtained from our PHANTOMS.zip with dcm2niix4pet.

In [11]:
# Check dcm2niix4pet help/usage options
import subprocess
result = subprocess.run(["dcm2niix4pet", "--help"], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

usage: dcm2niix4pet [-h] [--metadata-path [METADATA_PATH]]
                    [--translation-script-path TRANSLATION_SCRIPT_PATH]
                    [--destination-path DESTINATION_PATH] [--tempdir TEMPDIR]
                    [--kwargs [KWARGS [KWARGS ...]]] [--silent]
                    [--show-examples] [--set-dcm2niix-path SET_DCM2NIIX_PATH]
                    [--set-default-metadata-json SET_DEFAULT_METADATA_JSON]
                    [--trc TRC] [--run RUN] [--rec REC]
                    [folder]

Given a set of PET dicoms and additional metadata dcm2niix converts them to BIDS compliant nifti (using dcm2niix), json, and tsv files.

positional arguments:
  folder                Folder path containing imaging data

optional arguments:
  -h, --help            show this help message and exit
  --metadata-path [METADATA_PATH], -m [METADATA_PATH]
                        Path to metadata file for scan
  --translation-script-path TRANSLATION_SCRIPT_PATH, -t TRANSLATION_SCRIPT_PATH
  

In [12]:
# Run dcm2niix4pet to convert DICOMs to BIDS format (cross-platform)

input_dir = os.path.join(data_dir, "sourcedata", "SiemensBiographPETMR-NIMH", "AC_TOF")
output_dir = os.path.join(data_dir, "mynewfolder")
os.makedirs(output_dir, exist_ok=True)

print("Input directory contents:")
list_directory(input_dir)
print("Output directory will be:", output_dir)
print("dcm2niix path:", dcm2niix_path)

#cmd = ["dcm2niix4pet", input_dir, "--destination-path", output_dir,"--set-dcm2niix-path", dcm2niix_path]

!dcm2niix4pet "{input_dir}" --destination-path "{output_dir}" --set-dcm2niix-path "{dcm2niix_path}"

'''
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)
    '''

Input directory contents:
Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\sourcedata\SiemensBiographPETMR-NIMH\AC_TOF:
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0001.2021.05.04.09.18.07.924763.73825419.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0002.2021.05.04.09.18.07.924763.73825433.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0003.2021.05.04.09.18.07.924763.73825447.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0004.2021.05.04.09.18.07.924763.73825461.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0005.2021.05.04.09.18.07.924763.73825475.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0006.2021.05.04.09.18.07.924763.73825489.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0007.2021.05.04.09.18.07.924763.73825503.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_(ADULT).0003.0008.2021.05.04.09.18.07.924763.73825517.IMA
RESBRAIN_PHANTOM.PT.PET_PET_BRAIN_115MA_FDG_

'\nresult = subprocess.run(cmd, capture_output=True, text=True)\nprint(result.stdout)\nif result.stderr:\n    print("Error:", result.stderr)\n    '

In [13]:
# List files in the 'mynewfolder' directory (cross-platform)

list_directory(output_dir)

Contents of C:\Users\spn486\Downloads\PN000001\OpenNeuroPET-Phantoms\mynewfolder:


I could also add additional information regarding my data using additional flags:

In [14]:
# Run dcm2niix4pet with additional kwargs using Python subprocess (cross-platform)
import os
import subprocess

input_dir = os.path.join(data_dir, "sourcedata", "SiemensBiographPETMR-NIMH", "AC_TOF")
output_dir = "mynewfolder2"

kwargs = [
    "TimeZero=ScanStart",
    "Manufacturer=Siemens",
    "ManufacturersModelName=Biograph",
    "InstitutionName=NIH Clinical Center",
    "BodyPart=Phantom",
    "Units=Bq/mL",
    "TracerName=none",
    "TracerRadionuclide=F18",
    "InjectedRadioactivity=81.24",
    "SpecificRadioactivity=13019.23",
    "ModeOfAdministration=infusion",
    "FrameTimesStart=0",
    "AcquisitionMode=list mode",
    "ImageDecayCorrected=true",
    "ImageDecayCorrectionTime=0",
    "AttenuationCorrection=MR-corrected",
    "FrameDuration=300",
    "FrameTimesStart=0"
]

cmd = ["dcm2niix4pet", input_dir, "-d", output_dir, "--set-dcm2niix-path", dcm2niix_path, "--kwargs"] + kwargs
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

Now check if we have created a valid BIDS dataset

In [15]:
bids_validator('./mynewfolder2')

Hmm, what is wrong? Let's look at our file tree again.

In [16]:
!tree ./mynewfolder2

Folder PATH listing for volume Windows
Volume serial number is 0000009C FA70:B0C2
C:\USERS\SPN486\DOCUMENTS\GITHUB\OUTREACH\PETBIDS-ONBOARDING2025\MYNEWFOLDER2
Invalid path - \USERS\SPN486\DOCUMENTS\GITHUB\OUTREACH\PETBIDS-ONBOARDING2025\MYNEWFOLDER2
No subfolders exist 



Basically we have created the appropriate .nii.gz and .json files, but we haven't followed the proper BIDS naming convention!

We have done this for you here:

In [17]:
!tree ./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/ --filelimit 15

Too many parameters - --filelimit


Now let's validate this.



In [18]:
bids_validator('./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/')

Strictky speaking this only works since we actually have also added a dataset_description file to the file tree. See here:

In [19]:
!tree ./OpenNeuroPET-Phantoms/ --filelimit 15

Too many parameters - --filelimit


So even though there are libraries that do part of the BIDS conversion for you, there is often some manual work to be done wrt file renaming or adding additional files on top.

Converters like EZBids or BIDScoin try to even alleviate this burden.

We will play with EZBids!

Go to [EZBids](https://brainlife.io/ezbids/)